In [122]:
# sample codes for DF union

uall = net_addr[['u', 'uaddr']]
uall.columns = ['n', 'addr']
vall = net_addr[['v', 'vaddr']]
vall.columns = ['n', 'addr']

# UNION op
uv_union = pd.concat([uall, vall],
                     ignore_index=True).drop_duplicates().reset_index(drop=True)

#net_geo.insert(0, 'name', np.nan)

net_geo.loc[:, 'name'] = [uv_union.loc[uv_union.addr==a, 'n'].values[0] for a in net_geo.addr]

with pd.ExcelWriter('net.xlsx', engine='xlsxwriter') as writer:
    net_addr.to_excel(writer, index=False, sheet_name='addr')
    net_geo.to_excel(writer, index=False, sheet_name='geo')
    writer.save()

In [16]:
'''
Test sample up until now

START FROM HERE
'''
LINK_WUNIT = 60 # km

import pandas as pd
import numpy as np

net_addr = pd.read_excel('net.xlsx', sheet_name='addr')
net_geo = pd.read_excel('net.xlsx', sheet_name='geo')

In [17]:
import folium
from folium import plugins
folium.__version__

'0.10.1'

In [18]:
net_addr.head(3)

,type,u,v,uaddr,vaddr,distance,weight
0,M,서울(★서울청사),남양주(T중심국),서울 종로구 세종대로 209,경기 남양주시 금곡동 172-10,42,1
1,M,남양주(T중심국),S가평집중국,경기 남양주시 금곡동 172-10,경기 가평군 가평읍 읍내리 470-1,49,1
2,M,S가평집중국,춘천(강원도청),경기 가평군 가평읍 읍내리 470-1,강원 춘천시 중앙로 1,38,1


In [19]:
net_geo.head(3)

,name,addr,lat,lon
0,서울(★서울청사),서울 종로구 세종대로 209,37.574908,126.975207
1,남양주(T중심국),경기 남양주시 금곡동 172-10,37.637996,127.215612
2,S가평집중국,경기 가평군 가평읍 읍내리 470-1,37.830678,127.513030


In [32]:
# draw network topology

center = (37.541, 126.986) # center of seoul
m = folium.Map(location=center, zoom_start=7, tiles='cartodbpositron')

'''
plugins.Fullscreen(position='topright',
                   title='Full Screen',
                   title_cancel='Back to Normal',
                   force_separate_button=True).add_to(m)
'''

#m.add_child(folium.LatLngPopup())

fg = folium.FeatureGroup(name='links')
m.add_child(fg)

gm = plugins.FeatureGroupSubGroup(fg, 'main')
m.add_child(gm)
gx = plugins.FeatureGroupSubGroup(fg, 'mesh')
m.add_child(gx)
gr = plugins.FeatureGroupSubGroup(fg, 'sub')
m.add_child(gr)
    
# draw all links
linecolor = {'M' : 'crimson', 'X' : 'orange', 'N' : 'grey'}
g2add = {'M' : gm, 'X' : gx, 'N' : gr}
for i, uv in net_addr.iterrows():
    #uaddr, vaddr = uv['uaddr'], uv['vaddr']
    line = [
        net_geo.loc[net_geo.addr == uv['uaddr'], ['lat', 'lon']].values[0],
        net_geo.loc[net_geo.addr == uv['vaddr'], ['lat', 'lon']].values[0]
    ]
    
    folium.PolyLine(
        line,
        popup = str(uv['distance']),
        tooltip = str(uv['weight']) + ' links / ' + str(uv['distance']) + ' km',
        color = linecolor[uv['type']],
        weight = 3,
        dash_array = '1,5' if uv['weight'] > 1 else '1,1'
    ).add_to(g2add[uv['type']])
    
folium.LayerControl(collapsed=False).add_to(m)
    
# plot all nodes
for i, rgeo in net_geo.iterrows():
    folium.Circle(
        location = rgeo[['lat', 'lon']],
        popup = rgeo['name'],
        tooltip = rgeo['name'],
        radius = 250,
        color = 'darkblue',
        fill_color = 'crimson',
        weight = 5
    ).add_to(m)

In [33]:
m.save('net-map.html')

In [25]:
net_addr[net_addr.u == '안동(경북도청)']

,type,u,v,uaddr,vaddr,distance,weight
7,M,안동(경북도청),구미(구미시청),경북 안동시 풍천면 도청대로 455,경북 구미시 송정대로 55,107,2
51,N,안동(경북도청),포항(포항시청),경북 안동시 풍천면 도청대로 455,경북 포항시 남구 시청로 1,265,5
